<a href="https://colab.research.google.com/drive/1GHItQvWS9HBUoTmdZxDYnGq0wfmdYhsc?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://raw.githubusercontent.com/simvue-io/.github/refs/heads/main/simvue-white.png" />
    <source media="(prefers-color-scheme: light)" srcset="https://raw.githubusercontent.com/simvue-io/.github/refs/heads/main/simvue-black.png" />
    <img alt="Simvue" src="https://github.com/simvue-io/.github/blob/5eb8cfd2edd3269259eccd508029f269d993282f/simvue-black.png" width="500">
  </picture>
</p>

# Example tracking a Non-Python Simulation
This is an example of how you can use Simvue with Multiparser to track and monitor a non-Python simulation or other computational task. We will use a simple bash script to create some dummy 'temperature' data, looking at an experiment where the sample which is heated in an electric oven (causing its temperature to increase linearly), and then is taken out of the oven to cool down (losing temperature exponentially). We will then track this using the output file which is created during the experiment.

Note that for some common simulation softwares, we already have custom made integrations packages which can be used. These include:
* MOOSE (Multiphysics Object Oriented Simulation Environment)
* OpenFOAM
* FDS (Fire Dynamics Simulator)
* TensorFlow


### Install dependencies
While you can use Simvue and Multiparser individually to achieve the tracking and monitoring of Non-Python files, we recommend using the `simvue-integrations` package which wraps these both together into a handy `WrappedRun` class. Install this if you have not done so already:

In [5]:
!pip install git+https://github.com/simvue-io/integrations.git

  Cloning https://github.com/simvue-io/integrations.git to /tmp/pip-req-build-km5ajc3z
  Running command git clone --filter=blob:none --quiet https://github.com/simvue-io/integrations.git /tmp/pip-req-build-km5ajc3z
  Resolved https://github.com/simvue-io/integrations.git to commit b4975479df361af3d69a107389fe222b6022edd5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for simvue-integrations: filename=simvue_integrations-0.1.0-py3-none-any.whl size=31732 sha256=1d0cba85a517cbf653a6e0ab6df5a40371625b2f6035a47bc30daa36a5a96126
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9gjjxxn/wheels/92/f9/b9/eaf80712e9b7f8d7da9379e9b553361ffd45ae107842da9c28
Successfully built simvue-integrations



### Initialisation
To proceed you need to specify the URL of the Simvue server and provide an access token used to authenticate to the server. This can be done by either creating a `simvue.toml` file containing the required details, or specifying them as environment variables.

Login to https://app.simvue.io, go to the **Runs** page and click **Create new run**. Copy the 'token' from here. The run the cell below, paste the token into the box when prompted and push enter.

In [17]:
import os
import getpass

os.environ["SIMVUE_URL"] = "https://nightly.simvue.io"
os.environ["SIMVUE_TOKEN"] = getpass.getpass(prompt="Token: ")

### Using the WrappedRun class
To track our experiment, we will use the `add_process()` method to launch our bash script which generates the data, and we will use the `file_monitor` object to track our temperature data file. The WrappedRun class contains four methods which can be overridden to tailor to your needs:
* `_pre_simulation`: Things to do before any output files are tracked, such as uploading artifacts or launching the simulation
* `_during_simulation`: How to track the files created during the simulation and process data from them for upload to Simvue
* `_post_simulation`: Things to do after the simulation is complete, such as uploading final results files
* `launch`: Start the tracking session, calling each of the three methods above

In our case, we want to:
* Override `launch` to accept the path to the bash script
* Override `_pre_simulation` to upload and run the bash script
* Override `_during_simulation` to read from the temperature data as it is written and upload it as a metric

Note that `WrappedRun` inherits from Simvue's `Run` class, so contains all of the methods you are already familiar with such as `log_metrics`, `log_events` etc...

In [26]:
from simvue_integrations.connectors.generic import WrappedRun
import multiparser.parsing.tail as mp_tail_parser
import time
import pathlib

# Create a new class which inherits from WrappedRun
class TemperatureRun(WrappedRun):
    script_path: pathlib.Path = None
    
    # Override the `_pre_simulation` method to launch the process
    def _pre_simulation(self):
        # Call the base method first
        super()._pre_simulation()
        
        # Add a process to the run using `add_process`
        self.add_process(
            identifier="heating_experiment",
            executable="bash",
            script=self.script_path,
            completion_trigger=self._trigger # Sets a multiprocessing Event once the simulation is completed
        )
    
    # Override the `_during_simulation` method to track the temperature data
    def _during_simulation(self):
        # Use the `tail` method of the Multiparser `FileMonitor` object to track file, line by line
        self.file_monitor.tail(
            path_glob_exprs=str(self.script_path.with_suffix(".csv")),
            parser_func=mp_tail_parser.record_csv, # Use the built-in CSV parser, which returns a dictionary of data and metadata as each line is written
            callback=lambda csv_data, metadata: self.log_metrics( # Use data from those two dictionaries to log a metric:
                {'sample_temperature': csv_data["Temperature"]},
                 time=csv_data["Time"], 
                 step=csv_data["Step"], 
                 ) 
        )
    
    # Override the `launch` method to accept the path to the bash script
    def launch(self, script_path: str):
        self.script_path = script_path
        # Call the base `launch` method to call the above methods in the correct order
        super().launch()
                

### Using our TemperatureRun class
We can then use our `TemperatureRun` class in the same way as we would use the Simvue `Run` class - use it as a context manager, and call the `init` method. We can then add any additional information we want to store, before running the simulation by calling the `launch` method:

In [ ]:
with TemperatureRun() as run:
    run.init(
        name="heating-cooling-example-%d" % time.time(),
        folder="/examples",
        description="Simulate an experiment where a sample is heated and then left to cool, tracking the temperature.",
        tags=["example", "heating-cooling"],
    )
    # Can upload extra things we care about, eg could upload some metadata
    run.update_metadata(
        {
            "initial_temperature": 20,
            "heating_time": 50,
            "cooling_time": 100
        }
    )
    # Then run launch to start the experiment
    run.launch(pathlib.Path.cwd().joinpath("temperatures.sh"))
    
    # Then once complete, can upload any other information before closing the run
    run.save_file(pathlib.Path.cwd().joinpath("temperatures.csv"), category="output")

[simvue] Run heating-cooling-example-1740402163 created
[simvue] Monitor in the UI at https://nightly.simvue.io/dashboard/runs/run/CPnHkfbSYjHThHuyV2CazJ


ℹ️ | 1:17:53.596705  | INFO     | multiparser | Found NEW log '/home/wk9874/Documents/simvue/python-api/notebooks/temperatures.csv'
